# 2章：自然言語と単語の分散表現

#### ライブラリの読み込み

In [1]:
import numpy as np

## シソーラス
+ 基本的には類語辞書のことで、同義語や類義語を同じグループに分類したもの。
+ 自然言語処理におけるシソーラスでは「上位と下位」「全体と部分」の関係性をグラフ構造にて定義している。

### WordNet
+ 1985年にプリンストン大学で開発が進められた伝統的なシソーラス
+ 問題点
    + 単語の意味付けとして人の手でラベル付けしてしまうのは危険
    + 最新の造語などを絶えず更新する必要がある
    + 加えて人的コストが発生する
    + 単語の細かいニュアンス(使い方）を示すことができない
    
## コーパス
+ いわゆる大量のテキストデータ
     + 文章の書き方
     + 単語の選び方
     + 単語の意味 を含む

In [2]:
# コーパスとして利用するサンプル
text = "You say goodbye and I say hello."

### 単語単位に分割する

+ 小文字化には文頭の単語でも共通の単語落として扱えるようにするための処置

In [3]:
# 1.小文字化
text = text.lower()
# 2. ピリオドの分割
text = text.replace(".", " .")

In [4]:
text

'you say goodbye and i say hello .'

In [5]:
# textの分割
words = text.split(" ")

In [6]:
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

#### 正規表現による分割

In [7]:
import re
words = re.split("\W+", text)

In [8]:
# バージョンによる正規表現エンジンに問題あり？
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '']

### 単語をIDのリストとして対応表を作る

In [9]:
word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [10]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: ''}

In [11]:
word_to_id

{'': 6, 'and': 3, 'goodbye': 2, 'hello': 5, 'i': 4, 'say': 1, 'you': 0}

単語のリストを単語IDのリストへ変換

In [12]:
corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

これを関数化することによって前処理を一通り行うことができる。

In [13]:
def preprocess(text):
    # 1.小文字化
    text = text.lower()
    # 2. ピリオドの分割
    text = text.replace(".", " .")
    words = text.split(" ")
    
    word_to_id = {}
    id_to_word = {}

    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
            
    corpus = np.array([word_to_id[w] for w in words])
    
    return  corpus, word_to_id, id_to_word

## 単語のベクトル表現

In [14]:
def convert_one_hot(corpus, vocab_size):
    N = corpus.shape[0]
    one_hot = np.zeros((N, vocab_size), dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        one_hot[idx, word_id] = 1
        
    return one_hot

In [15]:
corpus = np.array([0,1,2,3,2])
vocab_size = 4

In [16]:
corpus_one_hot = convert_one_hot(corpus, vocab_size)

In [17]:
corpus_one_hot

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0]], dtype=int32)

one-hot表現で単語を固定長のベクトルとして扱うことができるが、**単語の保つ意味は一切ベクトルに加味されない**

## カウントベースの手法
### 分布仮説
+ 「単語の意味は、周辺単語によって形成される」
    + 単語自体に意味はなく、その単語の文脈によって意味が形成される 

### 分布仮説に基づいたベクトル化(共起行列)

In [18]:
import sys, os
sys.path.append(os.pardir)

text =  "You say goodbye and I say Python."
corpus, word_to_id, id_to_word = preprocess(text)

In [19]:
print(corpus)

[0 1 2 3 4 1 5 6]


In [20]:
print(id_to_word)

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'python', 6: '.'}


In [21]:
C = np.array([
    [0,1,0,0,0,0,0],
    [1,0,1,0,1,1,0],
    [0,1,0,1,0,0,0],
    [0,0,1,0,1,0,0],
    [0,1,0,1,0,0,0],
    [0,1,0,0,0,0,1],
    [0,0,0,0,0,1,0]
], dtype=np.int)

In [22]:
print(C[0])

[0 1 0 0 0 0 0]


In [23]:
print(C[word_to_id["goodbye"]])

[0 1 0 1 0 0 0]


### ベクトル間の類似度
ベクトル間の類似度については**コサイン類似度**をよく用いる

In [26]:
# ny=0のときに演算が失敗するため小さな値を設定する
def cos_similarity(x, y, eps=1e-8):
    nx = x / np.sqrt(np.sum(x**2) + eps)
    ny = y / np.sqrt(np.sum(y**2) + eps)
    return np.dot(nx, ny)

In [27]:
c0 = C[word_to_id["you"]]
c1 = C[word_to_id["i"]]
print(cos_similarity(c0, c1))

0.7071067758832467


### 類似単語のランキング表示

In [34]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for idx, word_idx in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i
            
            if left_idx >= 0:
                left_word_idx = corpus[left_idx]
                co_matrix[word_idx, left_word_idx] += 1
                
            if right_idx < corpus_size:
                right_word_idx = corpus[right_idx]
                co_matrix[word_idx, right_word_idx] += 1
                
    return co_matrix

In [28]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    # 1.queryの取り出し
    if query not in word_to_id:
        print("{} is not found".format(query))
        return
    
    print("\n[query] " + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]
    
    # 2.コサイン類似度の算出
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)
        
    # 3.コサイン類似度の結果から、その値を高い順に出力
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print("{}:{}".format(id_to_word[i], similarity[i]))
        
        count += 1
        if count >= top:
            return

In [32]:
x = np.array([100, -20, 2])
(-x).argsort()

array([0, 2, 1])

In [37]:
text = "You say perl and I say Python."
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
most_similar("you", word_to_id, id_to_word, C, top=5)


[query] you
perl:0.7071067758832467
i:0.7071067758832467
python:0.7071067758832467
say:0.0
and:0.0


## カウントベース手法の改善
+ 単純に出現回数だけを見てしまうと、本来の強い関連性と想定していない評価をされてしまう。
+ そこで相互情報量を定義する


In [38]:
def ppmi(C, verbose=False, eps=1e-8):
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    
    total = C.shape[0] * C.shape[1]
    cnt = 0
    
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i, j] * N / S[j] * S[i] + eps)
            M[i, j] = max(0, pmi)
            
            if verbose:
                cnt += 1
                if cnt % (total//100) == 0:
                    print("{0:02d} done".format(100*cnt/total))
                    
    return M

In [40]:
text = "You say perl and I say Python."
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
W = ppmi(C)

np.set_printoptions(precision=3)
print(C)
print("-"*50)
print(W)

[[0 1 0 0 0 0 0]
 [1 0 1 0 1 1 0]
 [0 1 0 1 0 0 0]
 [0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0]]
--------------------------------------------------
[[0.    1.807 0.    0.    0.    0.    0.   ]
 [5.807 0.    4.807 0.    4.807 4.807 0.   ]
 [0.    2.807 0.    3.807 0.    0.    0.   ]
 [0.    0.    3.807 0.    3.807 0.    0.   ]
 [0.    2.807 0.    3.807 0.    0.    0.   ]
 [0.    2.807 0.    0.    0.    0.    4.807]
 [0.    0.    0.    0.    0.    2.807 0.   ]]


### ベクトルの次元削減